In [1]:
import pandas as pd

In [79]:
matches=pd.read_csv("euros_matches.csv",index_col=0)

In [80]:
matches.shape

(780, 41)

In [81]:
matches.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'sot%', 'g/sh',
       'g/sot', 'dist', 'pk', 'pkatt', 'g-xg', 'np:g-xg', 'sca', 'gca',
       'att 3rd', 'att pen', 'att', 'succ', 'succ%', 'prgc', '1/3', 'cpa',
       'year', 'team'],
      dtype='object')

In [82]:
matches["comp"].value_counts()

UEFA Euro Qualifying    426
UEFA Euro               202
Friendlies (M)          152
Name: comp, dtype: int64

In [83]:
matches["year"].value_counts()

2024    390
2021    390
Name: year, dtype: int64

In [84]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf               object
ga               object
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes            object
sh              float64
sot             float64
sot%            float64
g/sh            float64
g/sot           float64
dist            float64
pk                int64
pkatt             int64
g-xg            float64
np:g-xg         float64
sca             float64
gca             float64
att 3rd         float64
att pen         float64
att             float64
succ            float64
succ%           float64
prgc            float64
1/3             float64
cpa             float64
year              int64
team             object
dtype: object

In [85]:
matches["venue"].head()

0       Away
1       Home
2       Home
3       Home
4    Neutral
Name: venue, dtype: object

In [86]:
matches["result"].value_counts()

W    435
D    177
L    168
Name: result, dtype: int64

In [87]:
matches["match report"].value_counts()

Match Report    780
Name: match report, dtype: int64

In [88]:
del matches["match report"]

In [89]:
matches["notes"].value_counts()

Required Extra Time                                      13
Italy won on penalty kicks following extra time           4
England won on penalty kicks following extra time         2
Portugal won on penalty kicks following extra time        2
France won on penalty kicks following extra time          2
Switzerland won on penalty kicks following extra time     2
Spain won on penalty kicks following extra time           2
Scotland won on penalty kicks following extra time        2
Poland won on penalty kicks following extra time          1
Match awarded to                                          1
Georgia won on penalty kicks following extra time         1
Slovakia won on penalty kicks following extra time        1
Name: notes, dtype: int64

In [90]:
del matches["notes"]

In [91]:
del matches["np:g-xg"]

In [92]:
matches["round"].value_counts()

Group stage       556
Friendlies (M)    152
Round of 16        32
Quarter-finals     16
Semi-finals        15
Finals              7
Final               2
Name: round, dtype: int64

In [93]:
# Convert date & time columns to datetime
matches['date'] = pd.to_datetime(matches['date'])
matches['time'] = pd.to_datetime(matches['time']).dt.time

In [94]:
matches.dtypes

date          datetime64[ns]
time                  object
comp                  object
round                 object
day                   object
venue                 object
result                object
gf                    object
ga                    object
opponent              object
xg                   float64
xga                  float64
poss                 float64
attendance           float64
captain               object
formation             object
referee               object
sh                   float64
sot                  float64
sot%                 float64
g/sh                 float64
g/sot                float64
dist                 float64
pk                     int64
pkatt                  int64
g-xg                 float64
sca                  float64
gca                  float64
att 3rd              float64
att pen              float64
att                  float64
succ                 float64
succ%                float64
prgc                 float64
1/3           

In [95]:
#Feature Engineering
#Assign competitions importance values, higher is more important
competition_map = {"UEFA Euro": 3,"UEFA Euro Qualifying": 2,"Friendlies (M)": 1}
matches['competition_importance'] = matches['comp'].map(competition_map).astype(float)

In [96]:
#Change final in round to finals
matches.loc[matches["round"] == "Final", "round"] = "Finals"

In [97]:
matches["round"].value_counts()

Group stage       556
Friendlies (M)    152
Round of 16        32
Quarter-finals     16
Semi-finals        15
Finals              9
Name: round, dtype: int64

In [98]:
#Feature Engineering
#Assign rounds importance values, higher is more important
round_map = {"Finals": 7,"Semi-finals": 6,"Quarter-finals": 5,"Round of 16": 4,"Group stage": 2, "Friendlies (M)": 1}
matches['round_importance'] = matches['round'].map(round_map).astype(float)

In [99]:
del matches["day"]

In [100]:
#Convert venue to a numerical category
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [101]:
matches["opponent"]=matches["opponent"].astype("category")

In [102]:
matches["gf"] = matches["gf"].str.replace("\(.+", "", regex=True)

In [103]:
matches["gf"].value_counts()

1      211
2      190
0      135
3      102
4       55
5       23
6       13
1        9
0        8
7        8
3.0      5
9        4
2.0      3
1.0      3
5.0      2
0.0      2
3        2
4.0      1
6.0      1
7.0      1
14       1
8        1
Name: gf, dtype: int64

In [104]:
matches["ga"] = matches["ga"].str.replace("\(.+", "", regex=True)

In [105]:
matches["ga"].value_counts()

0      314
1      249
2      113
3       45
4       16
1        9
0.0      8
1.0      8
0        8
5        4
3        2
2.0      1
3.0      1
6        1
7        1
Name: ga, dtype: int64

In [106]:
# Convert gf and ga to numeric
matches['gf'] = pd.to_numeric(matches['gf'])
matches['ga'] = pd.to_numeric(matches['ga'])

# Feature Engineering
# Goal Difference
matches['goal_difference'] = matches['gf'] - matches['ga']

# Expected Goals Difference
matches['xg_diff'] = matches['xg'] - matches['xga']

In [107]:
# Feature Engineering
# Calculate efficient possesion ratio
#https://breakingthelines.com/data-analysis/efficient-possession-ratio-a-new-football-performance-metric/#:~:text=The%20calculation%20of%20this%20metric,matches%2C%20thus%20creating%20a%20ratio.
matches['epr'] = matches['xg'] / matches['poss']

In [108]:
del matches['attendance']
del matches['referee']

In [109]:
# Feature Engineering
# Calculate total attacking touches in final 3rd
matches['att_touches']=matches['att 3rd']+matches['att pen']

In [110]:
del matches['att 3rd']
del matches['att pen']

In [111]:
matches.dtypes

date                      datetime64[ns]
time                              object
comp                              object
round                             object
venue                             object
result                            object
gf                               float64
ga                               float64
opponent                        category
xg                               float64
xga                              float64
poss                             float64
captain                           object
formation                         object
sh                               float64
sot                              float64
sot%                             float64
g/sh                             float64
g/sot                            float64
dist                             float64
pk                                 int64
pkatt                              int64
g-xg                             float64
sca                              float64
gca             

In [113]:
categorical_cols=['comp','round','formation','team']
for c in categorical_cols:
    matches[c] = matches[c].astype('category')

In [114]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [115]:
matches.columns

Index(['date', 'time', 'comp', 'round', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'captain', 'formation', 'sh', 'sot',
       'sot%', 'g/sh', 'g/sot', 'dist', 'pk', 'pkatt', 'g-xg', 'sca', 'gca',
       'att', 'succ', 'succ%', 'prgc', '1/3', 'cpa', 'year', 'team',
       'competition_importance', 'round_importance', 'venue_code',
       'goal_difference', 'xg_diff', 'epr', 'att_touches'],
      dtype='object')

In [119]:
cols = ["gf", "ga", "sh", "sot", "dist", "pk", "pkatt",'sot%', 'g/sh', 'g/sot','sca', 'gca',
       'att', 'succ', 'succ%', 'prgc', '1/3', 'cpa','goal_difference', 'xg_diff', 'epr', 'att_touches']
new_cols = [f"{c}_rolling" for c in cols]

In [120]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [121]:
matches_rolling = matches_rolling.droplevel('team')

In [123]:
# Adding the target column for multi-class classification
result_map = {'W': 1, 'D': 0, 'L': -1}
matches_rolling['target'] = matches_rolling['result'].map(result_map)

In [124]:
# Create dummy variables for categorical columns
matches_rolling = pd.get_dummies(matches, drop_first=True)

In [125]:
import ace_tools as tools; tools.display_dataframe_to_user(name="Preprocessed Matches Data with Imputed Values", dataframe=matches_rolling)

print(matches_rolling.info())

ModuleNotFoundError: No module named 'ace_tools'

In [128]:
matches_rolling['xg'].head(15)

0    NaN
1    NaN
2    NaN
3    NaN
4    2.1
5    1.4
6    1.6
7    2.6
8    2.1
0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
Name: xg, dtype: float64

In [137]:
matches_rolling.sort_values(by='date',inplace=True,ascending=True)

,date,gf,ga,xg,xga,poss,sh,sot,sot%,g/sh,...,team_Scotland,team_Serbia,team_Slovakia,team_Slovenia,team_Spain,team_Sweden,team_Switzerland,team_Turkiye,team_Ukraine,team_Wales
0,2019-03-21,3.0,1.0,NaN,NaN,60.0,25.0,8.0,32.0,0.12,...,0,0,0,0,0,0,0,0,0,0
0,2019-03-21,0.0,2.0,NaN,NaN,44.0,16.0,5.0,31.3,0.00,...,0,0,0,0,0,0,0,0,0,0
0,2019-03-21,1.0,3.0,NaN,NaN,39.0,4.0,2.0,50.0,0.25,...,0,0,0,0,0,0,0,0,0,0
0,2019-03-21,0.0,3.0,NaN,NaN,59.0,10.0,2.0,20.0,0.00,...,1,0,0,0,0,0,0,0,0,0
0,2019-03-21,3.0,1.0,NaN,NaN,61.0,15.0,6.0,40.0,0.13,...,0,0,0,0,0,0,0,0,0,0


In [138]:
matches_rolling.index = range(matches_rolling.shape[0])

In [139]:
matches_rolling.head()

,date,gf,ga,xg,xga,poss,sh,sot,sot%,g/sh,...,team_Scotland,team_Serbia,team_Slovakia,team_Slovenia,team_Spain,team_Sweden,team_Switzerland,team_Turkiye,team_Ukraine,team_Wales
0,2019-03-21,3.0,1.0,NaN,NaN,60.0,25.0,8.0,32.0,0.12,...,0,0,0,0,0,0,0,0,0,0
1,2019-03-21,0.0,2.0,NaN,NaN,44.0,16.0,5.0,31.3,0.00,...,0,0,0,0,0,0,0,0,0,0
2,2019-03-21,1.0,3.0,NaN,NaN,39.0,4.0,2.0,50.0,0.25,...,0,0,0,0,0,0,0,0,0,0
3,2019-03-21,0.0,3.0,NaN,NaN,59.0,10.0,2.0,20.0,0.00,...,1,0,0,0,0,0,0,0,0,0
4,2019-03-21,3.0,1.0,NaN,NaN,61.0,15.0,6.0,40.0,0.13,...,0,0,0,0,0,0,0,0,0,0


In [140]:
matches_with_xg = matches_rolling[matches_rolling['xg'].notna()]
matches_without_xg = matches_rolling[matches_rolling['xg'].isna()]

In [141]:
matches_with_xg.shape

(202, 298)

In [142]:
matches_without_xg.shape

(578, 298)

In [143]:
matches_rolling.columns

Index(['date', 'gf', 'ga', 'xg', 'xga', 'poss', 'sh', 'sot', 'sot%', 'g/sh',
       ...
       'team_Scotland', 'team_Serbia', 'team_Slovakia', 'team_Slovenia',
       'team_Spain', 'team_Sweden', 'team_Switzerland', 'team_Turkiye',
       'team_Ukraine', 'team_Wales'],
      dtype='object', length=298)

In [151]:
predictors=matches_rolling.columns.to_list()
predictors

['date',
 'gf',
 'ga',
 'xg',
 'xga',
 'poss',
 'sh',
 'sot',
 'sot%',
 'g/sh',
 'g/sot',
 'dist',
 'pk',
 'pkatt',
 'g-xg',
 'sca',
 'gca',
 'att',
 'succ',
 'succ%',
 'prgc',
 '1/3',
 'cpa',
 'year',
 'competition_importance',
 'round_importance',
 'venue_code',
 'goal_difference',
 'xg_diff',
 'epr',
 'att_touches',
 'target',
 'time_15:00:00',
 'time_16:00:00',
 'time_17:00:00',
 'time_17:30:00',
 'time_17:45:00',
 'time_18:00:00',
 'time_18:30:00',
 'time_18:45:00',
 'time_19:00:00',
 'time_19:30:00',
 'time_19:45:00',
 'time_20:00:00',
 'time_20:15:00',
 'time_20:30:00',
 'time_20:45:00',
 'time_21:00:00',
 'time_21:05:00',
 'time_21:10:00',
 'time_21:15:00',
 'time_21:30:00',
 'time_21:45:00',
 'time_22:00:00',
 'comp_UEFA Euro',
 'comp_UEFA Euro Qualifying',
 'round_Friendlies (M)',
 'round_Group stage',
 'round_Quarter-finals',
 'round_Round of 16',
 'round_Semi-finals',
 'venue_Home',
 'venue_Neutral',
 'result_L',
 'result_W',
 'opponent_al Albania',
 'opponent_am Armenia',


In [169]:
predictors.remove('date')
predictors.remove('year')

ValueError: list.remove(x): x not in list

In [210]:
matches_rolling.to_csv('filtered.csv')

In [ ]:
del matches_rolling['1/3']

In [236]:
train=matches_rolling[matches_rolling["date"] < '2024-06-13']
test=matches_rolling[matches_rolling["date"] > '2024-06-13']
test=test.dropna()

In [237]:
#train.to_csv('train.csv')
#test.to_csv('test.csv')
test.shape,train.shape

((96, 298), (680, 298))

In [238]:
matches_with_xg = train[train['xg'].notna()]
matches_without_xg = train[train['xg'].isna()]

matches_with_xg.shape, matches_without_xg.shape,matches_filtered.shape
#Drop columns where more than 20% of the data is NaN
threshold = 0.2 * len(matches_filtered)
matches_with_xg = matches_with_xg.dropna(thresh=threshold, axis=1)
matches_without_xg = matches_without_xg.dropna(thresh=threshold, axis=1)


In [239]:
matches_with_xg.shape, matches_without_xg.shape

((102, 298), (578, 283))

In [240]:
# Impute remaining NaN values in numerical columns with the mean
num_cols = matches_without_xg.select_dtypes(include=['float64', 'int64']).columns
matches_without_xg[num_cols] = matches_without_xg[num_cols].fillna(matches_without_xg[num_cols].mean())

num_cols = matches_with_xg.select_dtypes(include=['float64', 'int64']).columns
matches_with_xg[num_cols] = matches_with_xg[num_cols].fillna(matches_with_xg[num_cols].mean())

# Impute remaining NaN values in categorical columns with the mode
cat_cols = matches_without_xg.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    matches_without_xg[col].fillna(matches_without_xg[col].mode()[0], inplace=True)

cat_cols = matches_with_xg.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    matches_with_xg[col].fillna(matches_with_xg[col].mode()[0], inplace=True)


In [259]:
matches_with_xg.shape, matches_without_xg.shape

((102, 298), (578, 283))

In [241]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score,f1_score

In [250]:
predictors_1=matches_without_xg.columns.to_list()
predictors_1.remove('date')
predictors_1.remove('year')

predictors_2=matches_with_xg.columns.to_list()
predictors_2.remove('date')
predictors_2.remove('year')

In [251]:
# Define features and target for matches_without_xg
X_train_wo = matches_without_xg[predictors_1].drop(columns=['target'])
y_train_wo = matches_without_xg['target']

# Define features and target for matches_with_xg
X_train_w = matches_with_xg[predictors_2].drop(columns=['target'])
y_train_w = matches_with_xg['target']

# Define features and target for test set
X_test = test[predictors].drop(columns=['target'])
y_test = test['target']

In [260]:
X_train_wo.shape,y_train_wo.shape

((578, 280), (578,))

In [261]:
X_train_w.shape,y_train_w.shape

((102, 295), (102,))

In [262]:
X_test.shape,y_test.shape

((96, 295), (96,))

In [263]:
# Initialize models
rf = RandomForestClassifier(n_estimators=100, random_state=21)
clf = SVC(kernel='linear', random_state=21)

In [264]:
# Train and fine-tune the RandomForest model
rf.fit(X_train_wo, y_train_wo)
rf.fit(X_train_w, y_train_w)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,average='weighted')
f1 = f1_score(y_test, y_pred,average='weighted')
accuracy,precision,f1

(1.0, 1.0, 1.0)

In [265]:
# Train and fine-tune the SVC model
clf.fit(X_train_wo, y_train_wo)
clf.fit(X_train_w, y_train_w)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred,average='weighted')
f1 = f1_score(y_test, y_pred,average='weighted')
accuracy,precision,f1

(1.0, 1.0, 1.0)

In [268]:
rf.n_features_in_

295

KeyError: 0